In [1]:
import sympy as sp
from IPython.display import display, Math

In [2]:
# A = sp.Matrix([[1, 0, 1], \
#                [1, 1, 2], \
#                [0, 0, 1]])

A = sp.Matrix([[2, 0, 1, 0], \
               [0, 3, -1, 1], \
               [0, -1, 3, 0], \
               [0, 0, 0, 2]])

# A = sp.diag(sp.jordan_cell(-3, 3), sp.jordan_cell(-4, 2), 0, 0)
dim_A = A.shape[0]
A

Matrix([
[2,  0,  1, 0],
[0,  3, -1, 1],
[0, -1,  3, 0],
[0,  0,  0, 2]])

In [3]:
eigv = A.eigenvects()
num_eig = len(eigv)
eig_vals = [eigv[i][0] for i in range(num_eig)]
eig_amult = [eigv[i][1] for i in range(num_eig)]
eig_vecs = [eigv[i][2] for i in range(num_eig)]

In [4]:
for eig_num in range(num_eig):
    print("Eigenvalue: {}, Algebraic Multiplicity: {}".format(eig_vals[eig_num], eig_amult[eig_num]))
    print("Eigenvectors: ", end="")
    display(Math(r', '.join(['{}' for i in range(len(eig_vecs[eig_num]))]).format(*[sp.latex(elem) for elem in eig_vecs[eig_num]])))

Eigenvalue: 2, Algebraic Multiplicity: 3
Eigenvectors: 

<IPython.core.display.Math object>

Eigenvalue: 4, Algebraic Multiplicity: 1
Eigenvectors: 

<IPython.core.display.Math object>

In [5]:
J_assy = []
P_assy = []
# For printing the structure of P
struct_P = []

for eig_num in range(len(eig_vals)):
    r = [len(eig_vecs[eig_num])]
    g_evecs = {1: eig_vecs[eig_num]}
    g_order = 1
    g_A = [(A - eig_vals[eig_num]*sp.eye(dim_A))**g_order]
    while r[-1] < eig_amult[eig_num]:
        g_order += 1
        g_A.append(g_A[-1]**g_order)
        g_evecs[g_order] = g_A[-1].eigenvects()[0][2]
        r.append(len(g_evecs[g_order]))
    s = [r[0], *[r[j+1]-r[j] for j in range(len(r)-1)]]
    m = [*[s[j]-s[j+1] for j in range(len(r)-1)], s[-1]]
    dim_J_blks = [j+1 for j in range(len(m)) for i in range(m[j])]
    dim_J_blks.reverse()
    
    # Printing
    print("Eigenvalue: {}, Algebraic Multiplicity: {}".format(eig_vals[eig_num], eig_amult[eig_num]), end="")
    for i in range(1, g_order+1):
        g_ev_list = ', '.join([sp.latex(ev) for ev in g_evecs[i]])
        display(Math(r'\mathcal{{N}}((A - ({})I)^{}) = \text{{span}}\left( {}\right)'.format(eig_vals[eig_num], i, g_ev_list)))
    display(Math(r', '.join(['r_{}={}'.format(j+1, r[j]) for j in range(len(r))])))
    display(Math(r', '.join(['s_{}={}'.format(j+1, s[j]) for j in range(len(s))])))
    display(Math(r', '.join(['m_{}={}'.format(j+1, m[j]) for j in range(len(m))])))
    print(', '.join(["{} Jordan block(s) of size {}".format(m[j], j+1) for j in range(len(m)) if m[j] is not 0]))
    print("-----------------------------------------------------------------")
    
    J_blks = [sp.jordan_cell(eig_vals[eig_num], dim) for dim in dim_J_blks]
    J_assy.extend(J_blks)
    
    P_list = []
    zero_vec = sp.Matrix([0 for i in range(dim_A)])
    
    for idx, N in enumerate(dim_J_blks):
        temp_P = []
        vec_eq = zero_vec
        if P_list:
            a = sp.symbols('a0:{}'.format(len(P_list)))
            vec_eq = sum([a[j]*P_list[j] for j in range(1, len(a))], a[0]*P_list[0])
        for vec in g_evecs[N]:
            if N is 1:
                temp_P.append(vec)
                if P_list and sp.solve(vec_eq - temp_P[-1]):
                    temp_P = []
                    continue
                break
            if g_A[N-2]*vec != zero_vec:
                temp_P.append(vec)
                if P_list and sp.solve(vec_eq - temp_P[-1]):
                        temp_P = []
                else:
                    while g_A[0]*temp_P[-1] != zero_vec:
                        temp_P.append(g_A[0]*temp_P[-1])
                        if sp.solve(vec_eq - temp_P[-1]):
                            temp_P = []
                            break
                if temp_P:
                    break
        temp_P.reverse()
        P_list.extend(temp_P)
        
        # For printing
        struct_P.extend([r'v_{{{}, {} }}^{{{}}}'.format(idx+1, len(temp_P)-(j), eig_vals[eig_num]) for j in range(len(temp_P))])
        
    P_assy.extend(P_list)

Eigenvalue: 2, Algebraic Multiplicity: 3

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

1 Jordan block(s) of size 1
-----------------------------------------------------------------


In [6]:
display(Math(r'J=\text{{diag}}\left({}\right)'.format(', '.join([sp.latex(blk) for blk in J_assy]))))

<IPython.core.display.Math object>

In [7]:
J = sp.diag(*J_assy)
J

Matrix([
[2, 1, 0, 0],
[0, 2, 1, 0],
[0, 0, 2, 0],
[0, 0, 0, 4]])

In [8]:
display(Math(r'P=\begin{{bmatrix}}{}\end{{bmatrix}}'.format('&'.join(struct_P))))

<IPython.core.display.Math object>

In [9]:
P = sp.Matrix([*[col_P.T for col_P in P_assy]]).T
P

Matrix([
[1/2,   0,    0, 1/2],
[  0, 1/2, -1/2,  -1],
[  0, 1/2,    0,   1],
[  0,   0,    1,   0]])

In [10]:
P*J*P.inv()

Matrix([
[2,  0,  1, 0],
[0,  3, -1, 1],
[0, -1,  3, 0],
[0,  0,  0, 2]])

In [11]:
A

Matrix([
[2,  0,  1, 0],
[0,  3, -1, 1],
[0, -1,  3, 0],
[0,  0,  0, 2]])

In [12]:
P = sp.Matrix([[-1, 0, 0, 2], [0, -1, 1, -1], [0, -1, 0, 1], [0, 0, -2, 0]])
P

Matrix([
[-1,  0,  0,  2],
[ 0, -1,  1, -1],
[ 0, -1,  0,  1],
[ 0,  0, -2,  0]])

In [13]:
J = sp.diag(sp.jordan_cell(2, 3), 4)
J

Matrix([
[2, 1, 0, 0],
[0, 2, 1, 0],
[0, 0, 2, 0],
[0, 0, 0, 4]])

In [14]:
P*J*(P.inv())

Matrix([
[2, -3/2, 5/2, -3/4],
[0,    3,  -1,    1],
[0,   -1,   3,    0],
[0,    0,   0,    2]])